In [ ]:
conda run -n CueNote pip uninstall -y torch
conda run -n CueNote pip install torch --index-url https://download.pytorch.org/whl/cu121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 19.4 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 7.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.6 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 10.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.4/800.4 kB 8.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [transformers] [transformers]ub]


In [2]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)


torch: 2.5.1+cu121
cuda available: True
cuda device: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
pip install -U bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.1 MB/s  0:00:05m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
from typing import Any, Dict, Optional

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

#MODEL_DIR = "./data/Phi-3.5-mini-instruct"   # 또는 "./data/SmolLM2-1.7B-Instruct"
MODEL_DIR = "./data/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    device_map="cuda",          # ✅ 아예 GPU 고정 (offload 방지)
    load_in_4bit=True,          # ✅ 4bit 양자화
    torch_dtype=torch.float16,
)

def extract_first_json_object(text: str) -> str:
    """
    LLM 출력에서 '첫 번째' JSON 객체({ ... })만 중괄호 균형 기준으로 추출.
    JSON 앞/뒤에 다른 텍스트가 있어도 안전.
    """
    start = text.find("{")
    if start == -1:
        raise ValueError("No '{' found in model output.")

    depth = 0
    in_str = False
    escape = False

    for i in range(start, len(text)):
        ch = text[i]

        if in_str:
            if escape:
                escape = False
            elif ch == "\\":
                escape = True
            elif ch == '"':
                in_str = False
            continue

        if ch == '"':
            in_str = True
        elif ch == "{":
            depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0:
                return text[start:i + 1]

    raise ValueError("Unbalanced braces; could not extract JSON object.")

def parse_json_from_llm(text: str) -> Dict[str, Any]:
    json_str = extract_first_json_object(text)
    return json.loads(json_str)

def build_prompt(document: str, task: str) -> str:
    schema = {
        "task": task,
        "language": "ko",
        "output_schema": {
            "category": "one of [finance, legal, tech, medical, general, other]",
            "summary": "string (<= 120 chars)",
            "keywords": "array of strings (3~8 items)",
            "confidence": "number (0.0~1.0)"
        }
    }

    prompt = f"""
You are an on-device document understanding model.

STRICT RULES:
- Output MUST be valid JSON object.
- Output MUST contain ONLY the JSON object (no markdown, no extra text).
- Output MUST match the required schema keys exactly: category, summary, keywords, confidence.
- keywords MUST be a JSON array of 3~8 strings.
- confidence MUST be a number between 0.0 and 1.0.
- If uncertain, still output JSON with best guess and lower confidence.

REQUIRED JSON FORMAT EXAMPLE:
{{
  "category": "tech",
  "summary": "요약문...",
  "keywords": ["키워드1", "키워드2", "키워드3"],
  "confidence": 0.72
}}

TASK SPEC:
{json.dumps(schema, ensure_ascii=False)}

DOCUMENT:
{document}
""".strip()

    return prompt

def generate_text(prompt: str, max_new_tokens: int = 256) -> str:
    """
    중요: 모델 출력에서 '생성된 토큰만' 디코딩해서 프롬프트가 섞이지 않게 한다.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    # ✅ 프롬프트 길이만큼 잘라서 "새로 생성된 부분만" 디코딩
    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return text

def call_json(document: str, task: str = "classify_and_summarize") -> Dict[str, Any]:
    prompt = build_prompt(document, task)
    text = generate_text(prompt)

    try:
        return parse_json_from_llm(text)
    except Exception:
        # 1회 재시도: "JSON만" 수정하도록 강제
        repair_prompt = f"""
Your previous output was not valid JSON or contained extra text.
Return ONLY ONE valid JSON object that matches the schema.
No markdown. No explanations. No extra characters.

SCHEMA (keys must be exactly these):
- category: one of [finance, legal, tech, medical, general, other]
- summary: string (<= 120 chars)
- keywords: array of 3~8 strings
- confidence: number (0.0~1.0)

DOCUMENT:
{document}
""".strip()

        text2 = generate_text(repair_prompt, max_new_tokens=256)
        return parse_json_from_llm(text2)

# ---- 테스트
if __name__ == "__main__":
    doc = """현대자동차·기아의 자동차 부품을 운송하는 화물노동자 김아무개씨는 2024년 8월과 10월 배송 착오를 했다. 이로 인해 214분간 공장 생산라인이 중단됐다. 현대모비스 자회사인 부품업체 모트라스는 2억9천만원의 손해가 발생했다며 김씨와 위수탁계약을 맺은 운송 협력업체에 항의했다.

김씨는 모든 손해를 부담해야 했다. 운송업체는 김씨에게 책임을 돌리며, 지난해 6월 손해금을 김씨가 배상한다는 내용의 ‘라인중단 클레임 변제 합의서’를 김씨와 체결했기 때문이다. 김씨는 지입차를 팔아 1억원가량을 갚고, 모트라스·유니투스와 계약한 물류사의 직영기사로 고용돼 임금 중 일부를 떼어내 갚기로 합의했다. 지입차주에서 직영기사로 일하게 되며 한 달 보수가 절반으로 줄어드는 바람에 나머지 변제금은 보류 중이다.

특수고용 노동자라는 이유로 ‘무한책임’

이 같은 일은 단지 김씨만 겪고 있는 것이 아니다. 기아·현대자동차 화물운송 노동자 여러명이 비슷한 상태에 있다. 안현성 전국연대통합건설산업노조 화물운송분과 현대기아자동차 부품운송지부장은 6일 <매일노동뉴스>에 “전국 각지에 분당 손해비용을 계산해 몇백만 원, 몇천만 원, 1억원짜리 무한책임 변제를 강요하는 일들이 일어나고 있다”고 말했다.

화물노동자들에게 이 같은 책임이 지워지는 이유는 고용형태 때문이다. 이들은 계약상 특수고용노동자 신분이기 때문에 개인사업자로 분류된다. 이 때문에 부품운송이 지연돼 공장 생산라인이 지연되거나 중단되는 경우, 분당 거액의 비용이 ‘생산공정 중단 책임비용’이라는 이름으로 노동자들에게 부과된다. 노조에 따르면 현대자동차 아산공장 기준 책임비용은 1분당 152만8천원이다.

화물노동자들은 유한책임 제도로 바꿔야 한다고 요구하고 있다. 단순 실수 등으로 화물노동자의 삶이 ‘송두리째 뒤집어지는’ 일은 막아야 한다는 것이다. 이들은 현대차·기아, 현대모비스, 모트라스 소속 생산직 노동자들의 과실로 생산공정이 가동되지 않는 경우, 징계를 받을 수 있지만 분당 손해비용을 개인에게 부과하는 일이 없다는 사실을 강조했다.

“원청 교섭 나서지 않으면 9일 파업”

이들은 현대자동차·기아, 현대글로비스·현대모비스·모트라스·유니투스 등 원청사가 화물노동자와의 교섭에 나서라고 요구했다. 생산공정 중단 책임비용 청구는 현대자동차·기아에서부터 현대모비스·모트라스(유니투스)·현대글로비스·물류사를 거쳐 노동자에게 전가되는 구조기에, 이들 기업이 나서지 않으면 해결이 안 된다는 이유다.

개정 노동조합 및 노동관계조정법(노조법)에 따라 하청노동자가 원청사용자가 교섭할 수 있게 된 만큼, 실제로 원청과의 교섭이 이뤄질 가능성도 없지 않다. 지부는 부품을 싣고 내리는 특수 설비가 차량에 탑재돼 있고, 자동화 시스템을 통해 공장에 부품을 내리는 만큼 ‘하나의 사업’에서 일어나는 마지막 공정이라고 보고 있다.

지부에 법률자문을 하는 진우람 공인노무사(노동법률 신임)는 “배차 조정 등 업무지시나 운송료 결정 등에 있어 원청이 실질적 지배력이 있다는 정황증거가 존재하는 만큼 교섭 사용자로 인정될 수 있다고 본다”며 “물류업체와 교섭할 때 모트라스 등 원청 직원이 배석을 하고 있다. 자신들이 조건을 결정하는 지위에 있다는 것을 인지하고 있는 것”이라고 설명했다.

지부는 원청이 대화 의지를 밟히지 않으면 파업을 하겠다는 입장이다. 지부는 이날 오후 국회 소통관에서 기자회견을 열고 “원청과 계열사가 문제 해결에 나서지 못할 경우, 모트라스 직서열 차량과 글로비스 LST센터 소속 차량 등 200여대 규모의 연대 행동에 나설 수밖에 없다”며 “9일 쟁의행위를 실시하겠다”고 했다. 현대글로비스의 물류운송협력업체와 화물노동자들의 교섭은 지난달 31일 전남지방노동위원회에서 조정중지됐다. 모트라스의 물류운송협력업체와 화물노동자들의 교섭은 이달 2일 중앙노동위원회에서 조정중지됐다.
"""
    result = call_json(doc)
    print(json.dumps(result, ensure_ascii=False, indent=2))


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: No '{' found in model output.

In [2]:
if __name__ == "__main__":
    doc = "삼성전자는 차세대 반도체 공정 로드맵을 발표했으며, 2나노 공정 확대 계획을 밝혔다..."
    result = call_json(doc)
    print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "category": "tech",
  "summary": "삼성전자가 차세대 반도체 공정 로드맵을 발표하고 2나노 공정 확대 계획을 밝혔다.",
  "keywords": [
    "삼성전자",
    "반도체 공정",
    "2나노 공정",
    "로드맵"
  ],
  "confidence": 0.95
}
